In [1]:
library(SPATA2)
library(SPATAwrappers)
library(tidyverse)
# library(future)
# plan(multisession , workers = 4)
# options(future.globals.maxSize= 160*1024^3)



Attaching package: ‘SPATAwrappers’


The following objects are masked from ‘package:SPATA2’:

    hlpr_normalize_imap, hlpr_smooth_shiny


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::%||%()   masks SPATAwrappers::%||%()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 8 (Core)

Matrix products: default
BLAS/LAPACK: /data/person/lab/fengcz/conda/envs/spata2/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=C.UTF-8    LC_NUMERIC=C        LC_TIME=C          
 [4] LC_COLLATE=C        LC_MONETARY=C       LC_MESSAGES=C      
 [7] LC_PAPER=C          LC_NAME=C           LC_ADDRESS=C       
[10] LC_TELEPHONE=C      LC_MEASUREMENT=C    LC_IDENTIFICATION=C

time zone: Asia/Shanghai
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] lubridate_1.9.3          forcats_1.0.0            stringr_1.5.1           
 [4] dplyr_1.1.4              purrr_1.0.2              readr_2.1.5             
 [7] tidyr_1.3.1              tibble_3.2.1             ggplot2_3.5.1           
[10] tidyverse_2.0.0          SPATAwrappers_0.0.0.9000 SPATA2_2.0.4  

# Load SPATA2 infercnv functions

In [3]:
#' @title Check sample input
#'
#' @description A member of the \code{adjusting-check_*()}-family. Takes a character
#' vector of sample names and checks which of these exist.
#'
#' Returns an adjusted sample-vector or raises an error.
#' @param object A valid spata-object.
#' @param of_sample This argument is currently inactive. It might be reactivated when
#' spata-objects can store more than one sample.
#' @param desired_length The length the input must have.
#'
#' @keywords internal
check_sample <- function(object,
                         of_sample = "",
                         desired_length = NULL,
                         ...){


  # 0. Default sample -------------------------------------------------------

  # 'of_sample' is currently not in use
  if(FALSE){

  confuns::is_vec(of_sample, mode = "character", ...)

  if(base::all(of_sample == "")){

    of_sample <- getSampleName(object)[1]

    if(base::length(getSampleName(object)) > 1){

      base::message(glue::glue("No sample specified. Defaulting to first sample: '{of_sample}'."))

    }

  }

  # 1. Check which samples are in the object --------------------------------

  if(!base::is.character(of_sample) | base::length(of_sample) == 0){

    stop("Please specify the sample with its name as a character vector of length > 0.")

  } else if(base::all(of_sample == "all")){

    of_sample <- getSampleNames(object = object)

    if(!base::is.null(desired_length) && base::length(of_sample) != desired_length){

      stop(stringr::str_c("Number of samples specified needs to be: ", desired_length, ". ",
                          "Setting 'of_sample' to 'all' results in ",
                          base::length(of_sample), " samples.", sep = ""))

    }

    return(of_sample)

  }  else if(!base::any(of_sample %in% object@samples)){

    stop("Could not find any of the specified samples in specified object.")

  } else if(base::any(of_sample %in% getSampleNames(object))){

    samples_found <- object@samples[object@samples %in% of_sample]

    if(base::length(samples_found) != base::length(of_sample)){

      not_found <- of_sample[!of_sample %in% object@samples]
      n_not_found <- base::length(not_found)

      not_found <- stringr::str_c(not_found, collapse = "', '")

      ref <- base::ifelse(n_not_found > 1, "samples", "sample")

      warning(glue::glue("Did not find {n_not_found} {ref}: {not_found}"))

    }

  }

  # -----

  # 2. Check if length of samples found coincides with desired length -------

  if(!base::is.null(desired_length) &&
     base::length(samples_found) != desired_length){

    base::stop(stringr::str_c("Number of samples specified needs to be:", desired_length, sep = " "))

  }

  }

  # -----

  return(getSampleNames(object))

}


setInitiationInfo <- function(object,
                              additional_input = list()){

  ce <- rlang::caller_env()

  init_fn <- rlang::caller_fn()

  init_frame <- base::sys.parent()

  init_call <- base::sys.call(which = init_frame)

  init_fn_name <- base::as.character(init_call)[1]

  init_args <-
    rlang::fn_fmls_names(fn = init_fn)

  init_args <- init_args[init_args != "..."]

  init_args_input <-
    purrr::map(
      .x = init_args,
      .f = function(arg){

        base::parse(text = arg) %>%
          base::eval(envir = ce)

      }
    ) %>%
    purrr::set_names(nm = init_args)

  init_args_input <-
    init_args_input[!base::names(init_args_input) %in% c("cds",  "coords_df", "count_mtr", "expr_mtr", "object", "seurat_object")]

  init_args_input <-
    c(init_args_input, additional_input)

  initiation_list <- list(
    init_fn = init_fn_name,
    input = init_args_input,
    time = base::Sys.time()
  )

  object@information$initiation <-
    initiation_list

  base::return(object)

}

initiateSpataObject_modified <- function(coords_df,
                                         count_mtr,
                                         feature_df = NULL,
                                         sample_name,
                                         image = NULL,
                                         directory_spata = NULL,
                                         directory_seurat = NULL,
                                         combine_with_wd = FALSE,
                                         gene_set_path = NULL,
                                         SCTransform = FALSE,
                                         NormalizeData = list(normalization.method = "LogNormalize", scale.factor = 1000),
                                         FindVariableFeatures = list(selection.method = "vst", nfeatures = 2000),
                                         ScaleData = TRUE,
                                         RunPCA = list(npcs = 60),
                                         FindNeighbors = list(dims = 1:30),
                                         FindClusters = list(resolution = 0.8),
                                         RunTSNE = TRUE,
                                         RunUMAP = list(dims = 1:30),
                                         verbose = TRUE){

    # 1. Control --------------------------------------------------------------
    confuns::give_feedback(
      msg = "Starting initiation",
      verbose = verbose
    )

    confuns::is_value(x = sample_name, mode = "character", ref = "sample_name")

    confuns::check_data_frame(
      df = coords_df,
      var.class = list("barcodes" = "character", "x" = c("double", "integer", "numeric"), "y" = c("double", "integer", "numeric")),
      ref = "coords_df"
    )

    if(!methods::is(count_mtr, "Matrix")){

      base::stop("'count_mtr'-input needs to be of type 'Matrix'.")

    } else if(base::is.null(base::colnames(count_mtr))){

      base::stop("'count_mtr'-input needs to have column names")

    } else if(base::is.null(base::rownames(count_mtr))){

      base::stop("'count_mtr'-input needs to have row names.")

    }

    barcodes_count_mtr <- base::colnames(count_mtr) %>% base::sort()
    barcodes_coords_df <- dplyr::pull(coords_df, var = "barcodes") %>% base::sort()

    # check identical barcodes
    if(!base::identical(barcodes_count_mtr, barcodes_coords_df)){

      base::stop("Barcodes of 'coords_df'-input and column names of 'count_mtr'-input need to be identical.")

    }


    # -----

    # 2. Passing data ---------------------------------------------------------

    counts <- count_mtr

    seurat_object <-
      Seurat::CreateSeuratObject(counts = counts, meta.data = feature_df)

    # processed_seurat_object <-
    #   process_seurat_object(
    #     seurat_object = seurat_object,
    #     SCTransform = SCTransform,
    #     NormalizeData = NormalizeData,
    #     FindVariableFeatures = FindVariableFeatures,
    #     ScaleData = ScaleData,
    #     RunPCA = RunPCA,
    #     FindNeighbors = FindNeighbors,
    #     FindClusters = FindClusters,
    #     RunTSNE = RunTSNE,
    #     RunUMAP = RunUMAP,
    #     verbose = verbose
    #   )


    # 3. Passing features and images ------------------------------------------

    spata_object <-
      transformSeuratToSpata(
        seurat_object = seurat_object,
        assay_name = "RNA",
        sample_name = sample_name,
        gene_set_path = gene_set_path,
        method = "single_cell",
        verbose = verbose
      )

    spata_object <-
      setCoordsDf(object = spata_object, coords_df = coords_df) %>%
      setImage(object = ., image = image)

    spata_object <- setInitiationInfo(spata_object)

    # 4. Save and return object -----------------------------------------------

    # save seurat
    if(base::is.character(directory_seurat)){

      spata_object <-
        base::tryCatch({

          spata_object_return <-
            saveCorrespondingSeuratObject(
              seurat_object = processed_seurat_object,
              object = spata_object,
              directory_seurat = directory_seurat,
              combine_with_wd = combine_with_wd
            )

          spata_object_return

        }, error = function(error){

          base::warning(glue::glue("Attempt to save seurat-object under '{directory_seurat}' failed with the following error message: {error}"))

          spata_object

        })

    }

    # save spata
    if(base::is.character(directory_spata)){

      spata_object <-
        base::tryCatch({

          spata_object_ret <-
            saveSpataObject(
              object = spata_object,
              directory_spata = directory_spata,
              combine_with_wd = combine_with_wd
            )

          spata_object_ret

        }, error = function(error){

          base::warning(glue::glue("Attempt to save spata-object under '{directory_spata}' failed with the following error message: {error}"))

          spata_object

        })

    }

    confuns::give_feedback(
      msg = "Initiation finished.",
      verbose = verbose
    )

    # -----

    base::return(spata_object)

}






check_availability <- function(test, ref_x, ref_fns){

  if(!base::isTRUE(test)){

    msg <- glue::glue("Could not find {ref_x}. Make sure to use {ref_fns}.")

    confuns::give_feedback(msg = msg, fdb.fn = "stop", with.time = FALSE)

  }

  base::invisible(TRUE)

}


hlpr_assign_arguments <- function(object){

  check_object(object)

  default_instructions <- getDefaultInstructions(object)

  ce <- rlang::caller_env()

  cfn <- rlang::caller_fn()

  cargs <- rlang::fn_fmls_names(fn = cfn)

  default_args <- cargs[cargs %in% methods::slotNames(default_instructions_object)]

  for(arg in default_args){

    arg_value <-
      base::parse(text = arg) %>%
      base::eval(envir = ce)

    if(base::is.null(arg_value)){

      arg_value <- methods::slot(default_instructions, name = arg)

      base::assign(
        x = arg,
        value = arg_value,
        envir = ce
      )
    }

  }

  base::invisible(TRUE)

}

default_colors <- viridis::viridis(15)

default_instructions_object <-
  methods::new(Class = "default_instructions",
               average_genes = FALSE,
               binwidth = 1,
               clrp = "milo",
               clrsp = "inferno",
               colors = default_colors,
               complete = TRUE,
               display_facets = TRUE,
               display_image = FALSE,
               display_labels = TRUE,
               display_legend = TRUE,
               display_points = FALSE,
               display_residuals = TRUE,
               display_title = FALSE,
               display_trajectory_parts = FALSE,
               max_adj_pval = 0.05,
               method_aggl = "ward.D",
               method_dist = "euclidean",
               method_de = "wilcox",
               method_dr = "umap",
               method_gs = "mean",
               method_hclust = "complete",
               method_ovl = "classic",
               method_padj = "fdr",
               min_lfc = 0,
               normalize = TRUE,
               n_highest_lfc = 50,
               n_lowest_pval = 50,
               n_pcs = 10,
               position = "fill",
               pt_alpha = 0.9,
               pt_clr = "lightgrey",
               pt_clrp = "milo",
               pt_clrsp = "inferno",
               pt_fill = "black",
               pt_shape = 21,
               pt_size = 2,
               pt_size_fixed = TRUE,
               relevel = FALSE,
               scales = "free",
               sgmt_clr = "black",
               sgmt_size = 2.5,
               show_rownames = FALSE,
               show_colnames = FALSE,
               smooth = TRUE,
               smooth_clr = "red",
               smooth_method = "loess",
               smooth_se = TRUE,
               smooth_span = 0.25,
               uniform_genes = "discard",
               verbose = TRUE)

#' @title Set cnv-results
#'
#' @inherit check_sample params
#' @inherit set_dummy details
#'
#' @param cnv_list The list containing the results from \code{runCnvAnalysis()}.
#'
#' @return An updated spata-object.
#' @export
#'

setCnvResults <- function(object, cnv_list, of_sample = NA){

  check_object(object)

  of_sample <- check_sample(object = object, of_sample = of_sample, of.length = 1)

  object@cnv[[of_sample]] <- cnv_list

  base::return(object)

}


#' @title Obtain copy-number-variations results
#'
#' @description Provides convenient access to the results of \code{runCnvAnalysis()}.
#'
#' @inherit check_sample params
#'
#' @return A named list.
#' @export
#'

getCnvResults <- function(object, of_sample = NA){

  check_object(object)

  of_sample <- check_sample(object = object, of_sample = of_sample, of.length = 1)

  res_list <- object@cnv[[of_sample]]

  check_availability(test = !base::identical(x = res_list, y = list()),
                     ref_x = "CNV results",
                     ref_fns = "function 'runCnvAnalysis()")

  base::return(res_list)

}



#' @title Obtain features names under which cnv-analysis results are stored.
#'
#' @description Returns a character vector of feature names referring to the
#' barcode-spots chromosomal gains and losses as computed by \code{runCnvAnalysis()}.
#'
#' @inherit check_sample params
#'
#' @return Character vector.
#' @export
#'

getCnvFeatureNames <- function(object, of_sample = NA){

  check_object(object)

  of_sample <- check_sample(object = object, of_sample = of_sample, of.length = 1)

  cnv_results <- getCnvResults(object = object, of_sample = of_sample)

  prefix <- cnv_results$prefix

  chromosomes <-
    cnv_results$regions_df %>%
    base::rownames() %>%
    stringr::str_remove_all(pattern = "p$|q$") %>%
    base::unique()

  cnv_feature_names <- stringr::str_c(prefix, chromosomes)

  base::return(cnv_feature_names)

}


hlpr_run_cnva_pca <- function(object, n_pcs = 30, of_sample = NA, ...){

  check_object(object)

  of_sample <- check_sample(object, of_sample = of_sample, desired_length = 1)

  cnv_res <- getCnvResults(object, of_sample = of_sample)

  cnv_mtr <- cnv_res$cnv_mtr

  pca_res <- irlba::prcomp_irlba(x = base::t(cnv_mtr), n = n_pcs, ...)

  pca_df <-
    base::as.data.frame(x = pca_res[["x"]]) %>%
    dplyr::mutate(barcodes = base::colnames(cnv_mtr), sample = {{of_sample}}) %>%
    dplyr::select(barcodes, sample, dplyr::everything()) %>%
    tibble::as_tibble()

  cnv_res$dim_red$pca <- pca_df

  object <- setCnvResults(object, cnv_list = cnv_res, of_sample = of_sample)

  base::return(object)


}

runCnvAnalysis_modified <- function(object,
                           ref_annotation = cnv_ref[["annotation"]], # data.frame denoting reference data as reference
                           ref_mtr = cnv_ref[["mtr"]], # reference data set of healthy tissue
                           ref_regions = cnv_ref[["regions"]], # chromosome positions
                           gene_pos_df = SPATA2::gene_pos_df,
                           directory_cnv_folder = "data-development/cnv-results", # output folder
                           directory_regions_df = NA, # deprecated (chromosome positions)
                           n_pcs = 30,
                           cnv_prefix = "Chr",
                           save_infercnv_object = TRUE,
                           verbose = TRUE,
                           of_sample = NA,
                           CreateInfercnvObject = list(ref_group_names = "ref"),
                           require_above_min_mean_expr_cutoff = list(min_mean_expr_cutoff = 0.1),
                           require_above_min_cells_ref = list(min_cells_per_gene = 3),
                           normalize_counts_by_seq_depth = list(),
                           anscombe_transform = list(),
                           log2xplus1 = list(),
                           apply_max_threshold_bounds = list(),
                           smooth_by_chromosome = list(window_length = 101, smooth_ends = TRUE),
                           center_cell_expr_across_chromosome = list(method = "median"),
                           subtract_ref_expr_from_obs = list(inv_log = TRUE),
                           invert_log2 = list(),
                           clear_noise_via_ref_mean_sd = list(sd_amplifier = 1.5),
                           remove_outliers_norm = list(),
                           define_signif_tumor_subclusters = list(p_val = 0.05, hclust_method = "ward.D2", cluster_by_groups = TRUE, partition_method = "qnorm"),
                           plot_cnv = list(k_obs_groups = 5, cluster_by_groups = TRUE, output_filename = "infercnv.outliers_removed", color_safe_pal = FALSE,
                                           x.range = "auto", x.center = 1, output_format = "pdf", title = "Outliers Removed")
                           ){

  # 1. Control --------------------------------------------------------------

  hlpr_assign_arguments(object)
  of_sample <- check_sample(object = object, of_sample = of_sample, of.lenght = 1)

  confuns::are_values(c("save_infercnv_object"), mode = "logical")

  confuns::check_directories(directories = directory_cnv_folder, type = "folders")

  if(!base::is.na(directory_regions_df)){

    base::message(
      "Redirecting input for argument 'directory_regions_df' (deprecated) to ",
      "argument 'ref_regions'. Please use 'ref_regions' instead."
    )

    ref_regions <- directory_regions_df

    base::warning(
      "The argument 'directory_regions_df' is deprecated in favor of 'ref_regions'. ",
      "See documentation for more details."
      )

  }

  # -----


  # 2. Data extraction ------------------------------------------------------

  # preparing object derived data
  count_mtr <- getCountMatrix(object = object, of_sample = of_sample)

  obj_anno <-
    getFeatureDf(object = object, of_sample = of_sample) %>%
    dplyr::select(barcodes, sample) %>%
    tibble::column_to_rownames(var = "barcodes")

  # reading and preparing reference data
  confuns::give_feedback(msg = "Checking input for reference data.", verbose = verbose)

  if(base::is.character(ref_mtr) && stringr::str_detect(ref_mtr, pattern = "\\.RDS$")){

    confuns::give_feedback(
      msg = glue::glue("Reading in reference matrix from directory '{ref_mtr}'."),
      verbose = verbose
      )

    ref_mtr <- base::readRDS(file = ref_mtr)

    confuns::give_feedback(msg = "Done.", verbose = verbose)

  } else if(base::is.matrix(ref_mtr)){

    ref_mtr <- ref_mtr

  } else {

    base::stop("Input for argument 'ref_mtr' must either be a directory leading to an .RDS-file or a matrix.")

  }


  if(base::is.character(ref_annotation) && stringr::str_detect(ref_annotation, pattern = "\\.RDS$")){

    confuns::give_feedback(
      msg = glue::glue("Reading in reference annotation from directory '{ref_annotation}'."),
      verbose = verbose
    )

    ref_anno <- base::readRDS(file = ref_annotation)

    confuns::give_feedback(msg = "Done.", verbose = verbose)

  } else if(base::is.data.frame(ref_annotation)){

    ref_anno <- ref_annotation

  } else {

    base::stop("Input for argument 'ref_annotation' must either be a directory leading to an .RDS-file or a data.frame.")

  }


  # combine data sets
  confuns::give_feedback(msg = "Combining input and reference data.", verbose = verbose)

  genes_inter <-
    base::intersect(x = base::rownames(count_mtr), y = base::rownames(ref_mtr)) %>% base::unique()

  if(base::length(genes_inter) < 500){

    msg <- "Less than 500 genes match ref and count matrix."

    confuns::give_feedback(msg = msg, fdb.fn = "stop", with.time = FALSE)

  }

  expr_inter <- base::cbind(count_mtr[genes_inter, ], ref_mtr[genes_inter, ])

  anno_inter <- base::rbind(obj_anno, ref_anno)

  # read and process gene positions
  confuns::give_feedback(msg = "Getting gene positions.", verbose = verbose)

  if(base::is.character(ref_regions) && stringr::str_detect(ref_regions, pattern = "\\.RDS$")){

    regions_df <- base::readRDS(file = ref_regions)

  } else if(base::is.data.frame(ref_regions)) {

    regions_df <- ref_regions

  } else {

    base::stop("Input for argument 'ref_regions' must either be a directory leading to an .RDS-file or a data.frame")

  }

  if(base::is.data.frame(gene_pos_df)){

    confuns::check_data_frame(
      df = gene_pos_df,
      var.class = list(
        # ensembl_gene_id = "character",
        hgnc_symbol = "character",
        chromosome_name = "character",
        start_position = "integer",
        end_position = "integer"
      )
    )


  } else {

    gene_pos_df <-
      CONICSmat::getGenePositions(gene_names = base::rownames(expr_inter))

  }


  # -----


  # 3. Analysis pipeline ----------------------------------------------------

  gene_order_df <-
    dplyr::select(gene_pos_df, chromosome_name, start_position, end_position, hgnc_symbol) %>%
    magrittr::set_rownames(value = gene_pos_df$hgnc_symbol)

  confuns::give_feedback(msg = "Starting analysis pipeline.", verbose = verbose)

  infercnv_obj <-
    confuns::call_flexibly(
      fn = "CreateInfercnvObject",
      fn.ns = "infercnv",
      fn.ns.sep = ":::",
      default = list("raw_counts_matrix" = expr_inter,
                     "annotations_file" = anno_inter,
                     "gene_order_file" = gene_order_df
                     )
    )


  confuns::give_feedback(msg = glue::glue("Removing genes from matrix with mean expression below threshold."), verbose = verbose)

  infercnv_obj <-
    confuns::call_flexibly(
      fn = "require_above_min_mean_expr_cutoff",
      fn.ns = "infercnv",
      fn.ns.sep = ":::",
      default = list("infercnv_obj" = infercnv_obj),
      v.fail = infercnv_obj
      )

  confuns::give_feedback(msg = "Removing low quality barcode spots.", verbose = verbose)

  infercnv_obj <-
    confuns::call_flexibly(
      fn = "require_above_min_cells_ref",
      fn.ns = "infercnv",
      fn.ns.sep = ":::",
      default = list("infercnv_obj" = infercnv_obj),
      v.fail = infercnv_obj
    )


  confuns::give_feedback(msg = "Normalizing counts by sequencing depth.", verbose = verbose)

  infercnv_obj <-
    confuns::call_flexibly(
      fn = "normalize_counts_by_seq_depth",
      fn.ns = "infercnv",
      fn.ns.sep = ":::",
      default = list("infercnv_obj" = infercnv_obj),
      v.fail = infercnv_obj
    )


  confuns::give_feedback(msg = "Conducting anscombe and logarithmic transformation.", verbose = verbose)

  infercnv_obj <-
    confuns::call_flexibly(
      fn = "anscombe_transform",
      fn.ns = "infercnv",
      fn.ns.sep = ":::",
      default = list("infercnv_obj" = infercnv_obj),
      v.fail = infercnv_obj
    )

  infercnv_obj <-
    confuns::call_flexibly(
      fn = "log2xplus1",
      fn.ns = "infercnv",
      fn.ns.sep = ":::",
      default = list("infercnv_obj" = infercnv_obj),
      v.fail = infercnv_obj
    )

  confuns::give_feedback(msg = "Applying maximal threshold bounds.", verbose = verbose)

  # threshold <-
  #   base::mean(base::abs(infercnv:::get_average_bounds(infercnv_obj))) %>%
  #   base::round(digits = 2)

  threshold = 2.6
  infercnv_obj <-
    confuns::call_flexibly(
      fn = "apply_max_threshold_bounds",
      fn.ns = "infercnv",
      fn.ns.sep = ":::",
      default = list("infercnv_obj" = infercnv_obj, "threshold" = threshold),
      v.fail = infercnv_obj
    )


  confuns::give_feedback(msg = "Smoothing by chromosome.", verbose = verbose)

  infercnv_obj <-
    confuns::call_flexibly(
      fn = "smooth_by_chromosome",
      fn.ns = "infercnv",
      fn.ns.sep = ":::",
      default = list("infercnv_obj" = infercnv_obj),
      v.fail = infercnv_obj
    )


  confuns::give_feedback(msg = "Centering cell expression across chromosome.", verbose = verbose)

  infercnv_obj <-
    confuns::call_flexibly(
      fn = "center_cell_expr_across_chromosome",
      fn.ns = "infercnv",
      fn.ns.sep = ":::",
      default = list("infercnv_obj" = infercnv_obj),
      v.fail = infercnv_obj
    )

  confuns::give_feedback(msg = "Subtracting reference expression from observed expression.", verbose = verbose)

  infercnv_obj <-
    confuns::call_flexibly(
      fn = "subtract_ref_expr_from_obs",
      fn.ns = "infercnv",
      fn.ns.sep = ":::",
      default = list("infercnv_obj" = infercnv_obj),
      v.fail = infercnv_obj
    )

  confuns::give_feedback(msg = "Clearing noise via reference mean standard deviation.", verbose = verbose)

  infercnv_obj <-
    confuns::call_flexibly(
      fn = "invert_log2",
      fn.ns = "infercnv",
      fn.ns.sep = ":::",
      default = list("infercnv_obj" = infercnv_obj),
      v.fail = infercnv_obj

    )

  infercnv_obj <-
    confuns::call_flexibly(
      fn = "clear_noise_via_ref_mean_sd",
      fn.ns = "infercnv",
      fn.ns.sep = ":::",
      default = list("infercnv_obj" = infercnv_obj),
      v.fail = infercnv_obj
    )


  confuns::give_feedback(msg = "Removing outliers.", verbose = verbose)

  infercnv_obj <-
    confuns::call_flexibly(
      fn = "remove_outliers_norm",
      fn.ns = "infercnv",
      fn.ns.sep = ":::",
      default = list("infercnv_obj" = infercnv_obj),
      v.fail = infercnv_obj
    )


#   confuns::give_feedback(msg = "Defining significant tumor subclusters.", verbose = verbose)

#   infercnv_obj <-
#     confuns::call_flexibly(
#       fn = "define_signif_tumor_subclusters",
#       fn.ns = "infercnv",
#       fn.ns.sep = ":::",
#       default = list("infercnv_obj" = infercnv_obj),
#       v.fail = infercnv_obj
#     )


  confuns::give_feedback(msg = "Copy number variation pipeline completed.", verbose = verbose)

  if(base::isTRUE(save_infercnv_object)){

    save_dir <- stringr::str_c(directory_cnv_folder, "infercnv-obj.RDS", sep = "/")

    msg <- glue::glue("Saving infercnv-object under '{save_dir}'.")

    confuns::give_feedback(msg = msg, verbose = verbose)

    base::saveRDS(infercnv_obj, file = save_dir)

  }

#   confuns::give_feedback(msg = "Plotting results.", verbose = verbose)

#   plot_results <-
#     confuns::call_flexibly(
#       fn = "plot_cnv",
#       fn.ns = "infercnv",
#       fn.ns.sep = ":::",
#       default = list("infercnv_obj" = infercnv_obj, "out_dir" = directory_cnv_folder),
#       v.fail = NULL
#     )


#   # work around a weird error in plot_cnv()
#   if(base::is.null(plot_results)){

#     confuns::give_feedback(
#       msg = "infercnv:::plot_cnv() failed. Attempting to plot with default setting.",
#       verbose = TRUE
#     )

#     plot_results <-
#       base::tryCatch({

#         infercnv::plot_cnv(infercnv_obj = infercnv_obj, out_dir = directory_cnv_folder)

#       }, error = function(error){

#         NULL

#       })

#     if(base::is.null(plot_results)){

#       confuns::give_feedback(
#         msg = "inferncnv::plot_cnv() failed with default setting, too.",
#         verbose = TRUE
#       )

#     }

#     if(base::isTRUE(save_infercnv_object)){

#       msg <-
#         glue::glue(
#           "The infercnv-object has been saved under '{save_dir}'.",
#           "Please try to plot the heatmap manually."
#           )

#       confuns::give_feedback(msg = msg, verbose = TRUE)

#     } else {

#       msg <-
#         glue::glue(
#           "Please consider to run runCnvAnalysis() again with argument 'save_infercnv_object' set to TRUE.",
#           "This way you can plot the results manually."
#         )

#       confuns::give_feedback(msg = msg, verbose = TRUE)

#     }

#     msg <- "If the error in infercnv:::plot_cnv() persists, consider to open an issue at https://github.com/theMILOlab/SPATA2/issues."

#     confuns::give_feedback(msg = msg, verbose = TRUE)

#   }

  # ----


  # 4. Storing results ------------------------------------------------------

  # result_dir <-
  #   stringr::str_c(directory_cnv_folder, "/", plot_cnv$output_filename, ".observations.txt")

    
  # infercnv_obj = readRDS(paste0(directory_cnv_folder,'/infercnv-obj.RDS'))

  # results <- utils::read.table(result_dir)
  results = infercnv_obj@expr.data
    
  barcodes <- base::colnames(results)

  confuns::give_feedback(msg = "Summarizing cnv-results by chromosome.", verbose = verbose)

  # join cnv results (per gene) with chromosome positions and summarize by chromosome
  ordered_cnv_df <-
    base::as.data.frame(results) %>%
    tibble::rownames_to_column("hgnc_symbol") %>%
    dplyr::left_join(., gene_pos_df, by = "hgnc_symbol") %>%
    dplyr::group_by(chromosome_name) %>%
    dplyr::select(chromosome_name, dplyr::any_of(barcodes)) %>%
    dplyr::summarise_all(base::mean) %>%
    dplyr::mutate(Chr = stringr::str_c(cnv_prefix, chromosome_name)) %>%
    dplyr::select(Chr, dplyr::everything()) %>%
    dplyr::ungroup() %>%
    dplyr::select(-chromosome_name)

  cnames <- c("barcodes", ordered_cnv_df$Chr)

  ordered_cnv_df2 <-
    dplyr::select(ordered_cnv_df, -Chr) %>%
    base::t() %>%
    base::as.data.frame() %>%
    tibble::rownames_to_column(var = "barcodes") %>%
    magrittr::set_colnames(value = cnames) %>%
    dplyr::mutate(barcodes = stringr::str_replace_all(string = barcodes, pattern = "\\.", replacement = "-")) %>%
    dplyr::mutate(dplyr::across(dplyr::starts_with(match = cnv_prefix), .fns = base::as.numeric)) %>%
    tibble::as_tibble()

  # add results to spata object
  confuns::give_feedback(msg = "Adding results to the spata-object's feature data.", verbose = verbose)

  # feature variables
  object <-
    addFeatures(
      object = object,
      feature_df = ordered_cnv_df2,
      overwrite = TRUE
      )

  # cnv matrix
  base::colnames(results) <-
    stringr::str_replace_all(
      string = base::colnames(results),
      pattern = "\\.",
      replacement = "-"
      )

  cnv_mtr <- base::as.matrix(results)

  # cnv list
  cnv_list <-
    list(
      prefix = cnv_prefix,
      cnv_df = ordered_cnv_df2,
      cnv_mtr = cnv_mtr,
      gene_pos_df = gene_pos_df,
      regions_df = regions_df
      )

  object <-
    setCnvResults(object = object, cnv_list = cnv_list, of_sample = of_sample)

#   confuns::give_feedback(msg = "Computing PCA based on cnv results.", verbose = verbose)

#   object <-
#     hlpr_run_cnva_pca(object, n_pcs = n_pcs, of_sample = of_sample)

#   # cnv clustering - hierarchical
#   cnv_res <- getCnvResults(object, of_sample = of_sample)

#   cnv_pca_df <- dplyr::select(cnv_res$dim_red$pca, -sample)

#   cnv_hclust <-
#     confuns::initiate_hclust_object(
#       hclust.data = tibble::column_to_rownames(cnv_pca_df, var = "barcodes"),
#       key.name = "barcodes"
#       )

#   clustering_list <- list(hierarchical = cnv_hclust)

#   cnv_res$clustering <- clustering_list

#   object <- setCnvResults(object, cnv_list = cnv_res, of_sample = of_sample)

  # -----
    
    
  confuns::give_feedback(msg = "Infercnv Done.", verbose = verbose)

  base::return(object)

}




# Read data

In [4]:
P0_FT_1 = readRDS('./P0_FT_1.sce.rds')
P0_FT_1

class: SingleCellExperiment 
dim: 16147 38012 
metadata(0):
assays(1): counts
rownames(16147): A1BG A2M ... ZYX ZZEF1
rowData names(0):
colnames(38012): P0_FT_1_50_7550 P0_FT_1_50_7750 ... P0_FT_1_9150_9150
  P0_FT_1_4900_8700
colData names(35): n_genes_by_counts log1p_n_genes_by_counts ... myCAF
  MMP11- Cell_type_refined
reducedDimNames(1): spatial
mainExpName: NULL
altExpNames(0):

In [5]:
P6_YF_1 = readRDS('./P6_YF_1.sce.rds')
P6_YF_1

class: SingleCellExperiment 
dim: 16147 87959 
metadata(0):
assays(1): counts
rownames(16147): A1BG A2M ... ZYX ZZEF1
rowData names(0):
colnames(87959): P6_YF_1_0_2550 P6_YF_1_0_3250 ... P6_YF_1_7950_15350
  P6_YF_1_8050_15400
colData names(35): n_genes_by_counts log1p_n_genes_by_counts ... myCAF
  MMP11- Cell_type_refined
reducedDimNames(1): spatial
mainExpName: NULL
altExpNames(0):

In [6]:
cnv_ref = SPATA2::cnv_ref

In [7]:
gene_pos_df = read.table('gene_pos.txt',sep='\t')
colnames(gene_pos_df) = c('hgnc_symbol','chromosome_name','start_position','end_position')
gene_pos_df$chromosome_name = gsub('chr','',gene_pos_df$chromosome_name)

# Run inferCNV

In [9]:
runInferCNV = function(sce,sample){

    # Create SPATA object
    
    spata_obj = initiateSpataObject_Empty(sample)
    spata_obj@data[[sample]]$counts = Matrix::Matrix(sce@assays@data$counts[,sce$Sample==sample])
    spata_obj@fdata[[sample]] = cbind(sce@colData[sce$Sample==sample,] %>% as.data.frame() %>% rownames_to_column('barcodes'),'sample'=sample)
    spata_obj@coordinates[[sample]] = cbind(sce[,sce$Sample==sample]@int_colData$reducedDims$spatial,'sample'=sample)
    
    spata_obj <-
        setDefaultInstructions(spata_obj) %>%
        setDirectoryInstructions()

    spata_obj <- setInitiationInfo(spata_obj)
    
    if (!dir.exists(paste0('./window100/',sample))){
    dir.create(paste0('./window100/',sample),recursive = T)
    }

    # Run infercnv using SPATA2 functions
    spata_obj <-
      runCnvAnalysis_modified(
        object = spata_obj,
        directory_cnv_folder = paste0('./window100/',sample), # example directory
        gene_pos_df = gene_pos_df,
        cnv_prefix = "Chr",
        ref_annotation = data.frame('barcodes' = colnames(P0_FT_1),'sample'='ref') %>% column_to_rownames('barcodes'), 
        ref_mtr = as.matrix(P0_FT_1@assays@data$counts), 
        ref_regions = cnv_ref$regions,
        smooth_by_chromosome = list(window_length = 101, smooth_ends = TRUE),
        define_signif_tumor_subclusters = list(p_val = 0.05, hclust_method = "ward.D2",
            cluster_by_groups = FALSE, partition_method = "qnorm")
        )
    
    confuns::give_feedback(msg = "Trimming out ref in cnv matrix.", verbose = T)
    spata_obj@cnv[[sample]]$cnv_mtr = spata_obj@cnv[[sample]]$cnv_mtr[,colnames(spata_obj@cnv[[sample]]$cnv_mtr) %in% spata_obj@fdata[[sample]]$barcodes ]
    
    confuns::give_feedback(msg = "Saving SPATA object.", verbose = T)
    saveRDS(spata_obj,paste0('./window100/',sample,'/spata_obj.RDS'))

}

In [10]:
runInferCNV(P6_YF_1,'P6_YF_1')


14:18:56 Setting up new `spata2` object.

Warning message in confuns::give_feedback(msg = msg, fdb.fn = fdb_fn, with.time = FALSE):
“Argument `of_sample` is deprecated and no longer in use.( used by runCnvAnalysis_modified() )”
14:18:57 Checking input for reference data.

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 4.6 GiB”
14:19:00 Combining input and reference data.

14:19:04 Getting gene positions.

14:19:04 Starting analysis pipeline.

Warning message:
“`invoke()` is deprecated as of rlang 0.4.0.
Please use `exec()` or `inject()` instead.
This warning is displayed once every 8 hours.”


INFO [2025-11-21 14:19:05] ::order_reduce:Start.
INFO [2025-11-21 14:19:05] .order_reduce(): expr and order match.
INFO [2025-11-21 14:19:08] ::process_data:order_reduce:Reduction from positional data, new dimensions (r,c) = 16147,125971 Total=191114865 Min=0 Max=277.
INFO [2025-11-21 14:19:09] -filtering out cells < 100 or > Inf, removing 0.12622 % of cells
WARN [2025-11-21 14:19:10] Please use "options(scipen = 100)" before running infercnv if you are using the analysis_mode="subclusters" option or you may encounter an error while the hclust is being generated.
INFO [2025-11-21 14:19:13] validating infercnv_obj


14:19:14 Removing genes from matrix with mean expression below threshold.



INFO [2025-11-21 14:19:13] ::above_min_mean_expr_cutoff:Start
INFO [2025-11-21 14:19:14] Removing 12625 genes from matrix as below mean expr threshold: 0.1
INFO [2025-11-21 14:19:15] validating infercnv_obj
INFO [2025-11-21 14:19:15] There are 3522 genes and 125812 cells remaining in the expr matrix.


14:19:15 Removing low quality barcode spots.

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 3.3 GiB”


...

14:49:00 Clearing noise via reference mean standard deviation.



INFO [2025-11-21 14:49:00] invert_log2(), computing 2^x
INFO [2025-11-21 14:49:14] denoising using mean(normal) +- sd_amplifier * sd(normal) per gene per cell across all data
INFO [2025-11-21 14:49:18] :: **** clear_noise_via_ref_quantiles **** : removing noise between bounds:  0.966171457884711 - 1.03382854211529


14:49:27 Removing outliers.



INFO [2025-11-21 14:49:27] ::remove_outlier_norm:Start out_method: average_bound lower_bound: NA upper_bound: NA
INFO [2025-11-21 14:49:27] ::remove_outlier_norm:Start out_method: average_bound lower_bound: NA upper_bound: NA
INFO [2025-11-21 14:49:27] ::remove_outlier_norm using method: average_bound for defining outliers.
INFO [2025-11-21 14:50:33] outlier bounds defined between: 0.942583 - 1.08848


14:50:41 Copy number variation pipeline completed.

14:50:41 Saving infercnv-object under './window100/P6_YF_1/infercnv-obj.RDS'.

14:51:41 Summarizing cnv-results by chromosome.

15:10:43 Adding results to the spata-object's feature data.

Warning message in addFeatures(object = object, feature_df = ordered_cnv_df2, overwrite = TRUE):
“Added features contain data for 125812 barcodes. Spata object contains 87959. Missing barcodes get NAs as values.”
15:10:45 Infercnv Done.

15:10:45 Trimming out ref in cnv matrix.

15:10:46 Saving SPATA object.



# Convert to csv files to read in python

In [11]:
convert_spata = function(sample){
    spata_obj = readRDS(paste0('./window100/',sample,'/spata_obj.RDS'))
    write.csv(
        spata_obj@cnv[[sample]]$cnv_mtr[,colnames(spata_obj@cnv[[sample]]$cnv_mtr) %in% spata_obj@fdata[[sample]]$barcodes],
        paste0('./window100/',sample,'/cnv.csv')
    )

    metadata = spata_obj@fdata[[sample]] %>% column_to_rownames('barcodes') 
    metadata = metadata[colnames(spata_obj@cnv[[sample]]$cnv_mtr),]
    write.csv(metadata,paste0('./window100/',sample,'/metadata.csv'))
    
    coord = spata_obj@coordinates[[sample]]
    # rownames(coord) = coord$barcodes
    coord = coord[colnames(spata_obj@cnv[[sample]]$cnv_mtr),c('x','y')]
    write.csv(coord,paste0('./window100/',sample,'/coord.csv'))

    
    
}

In [12]:
convert_spata('P6_YF_1')